**NIH Chest Xray Data Analysis And Cleaning:**

This dataset is know to have poor data integrity. New folders will be created to review, re-interpreted, and relabel the images. These new dataset will be then be used to train an simple CNN model.

In [1]:
import pandas as pd
import os
import shutil
import splitfolders
import random

In [2]:
# create file_path and load CSV file
Images_PATH= "Documents/ML/CXR"
df= pd.read_csv("Data_Entry_2017.csv")
pd.set_option('display.max_rows', 100)
df


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN


**Data exploration: Select colomns.**

In [3]:
df= df[['Image Index','Finding Labels', 'Follow-up #', 'Patient Age','Patient Gender', 'View Position' ]]
df

,Image Index,Finding Labels,Follow-up #,Patient Age,Patient Gender,View Position
0,00000001_000.png,Cardiomegaly,0,58,M,PA
1,00000001_001.png,Cardiomegaly|Emphysema,1,58,M,PA
2,00000001_002.png,Cardiomegaly|Effusion,2,58,M,PA
3,00000002_000.png,No Finding,0,81,M,PA
4,00000003_000.png,Hernia,0,81,F,PA
...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,39,M,PA
112116,00030802_000.png,No Finding,0,29,M,PA
112117,00030803_000.png,No Finding,0,42,F,PA
112118,00030804_000.png,No Finding,0,30,F,PA


**Data exploration: Label categories and counts.**

In [ ]:
#Data exploration of finding labels
df['Finding Labels'].value_counts()

No Finding                                                    60361
Infiltration                                                   9547
Atelectasis                                                    4215
Effusion                                                       3955
Nodule                                                         2705
                                                              ...  
Cardiomegaly|Consolidation|Effusion|Pleural_Thickening            1
Cardiomegaly|Hernia|Infiltration                                  1
Atelectasis|Cardiomegaly|Consolidation|Edema|Effusion|Mass        1
Atelectasis|Edema|Effusion|Emphysema|Infiltration|Mass            1
Edema|Effusion|Pleural_Thickening|Pneumonia|Pneumothorax          1
Name: Finding Labels, Length: 836, dtype: int64

**Data exploration:**

**In clinical practice young adults have cleaner appearing chest xrays (with less chronic changes) and acute disease changes are prominent. Images with ages between 20-40 and PA view will be selected for new dataframe and new folders will be created from this dataframe.**

In [ ]:
# Create and view new dataframe
adult_20_40 = df[(df['Patient Age']>= 20) & (df['Patient Age'] <=40)]
adult_20_40.head(10)

,Image Index,Finding Labels,Follow-up #,Patient Age,Patient Gender,View Position
407,00000101_000.png,Nodule,0,33,M,PA
573,00000140_000.png,No Finding,0,25,F,PA
574,00000141_000.png,No Finding,0,31,M,PA
630,00000157_000.png,Nodule,0,40,F,PA
777,00000190_000.png,No Finding,0,30,M,PA
778,00000190_001.png,No Finding,1,31,M,PA
779,00000190_002.png,No Finding,2,33,M,PA
780,00000190_003.png,No Finding,3,32,M,PA
781,00000190_004.png,No Finding,4,32,M,PA
782,00000190_005.png,No Finding,5,34,M,PA


**Data exploration:
Find number of studies, any NaN and counts of respective finding labels.**

In [ ]:

adult_20_40.count()

Image Index       31131
Finding Labels    31131
Follow-up #       31131
Patient Age       31131
Patient Gender    31131
View Position     31131
dtype: int64

In [ ]:
adult_20_40.isnull().sum()

Image Index       0
Finding Labels    0
Follow-up #       0
Patient Age       0
Patient Gender    0
View Position     0
dtype: int64

In [ ]:
adult_20_40['Finding Labels'].value_counts()

No Finding                                        18006
Infiltration                                       2965
Effusion                                            881
Atelectasis                                         833
Pneumothorax                                        641
                                                  ...  
Atelectasis|Edema|Effusion|Infiltration|Mass          1
Fibrosis|Nodule|Pneumothorax                          1
Consolidation|Effusion|Mass|Pleural_Thickening        1
Consolidation|Edema|Effusion|Infiltration             1
Atelectasis|Edema|Infiltration|Pneumonia              1
Name: Finding Labels, Length: 459, dtype: int64

**Data exploration:
Find the count for the label "No Finding" and view PA**

In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "No Finding" and row['View Position']== 'PA':
        cnt += 1
print(cnt)

11849


**Create folder for images with labels "No Finding" and "PA" view for review.** 

In [ ]:
TARGET_DIR= 'Dataset/Normal'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

CXR folder created


In [ ]:
#Exporting images into new folder
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "No Finding" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

moving image 0
moving image 1
moving image 2
moving image 3
moving image 4
moving image 5
moving image 6
moving image 7
moving image 8
moving image 9
moving image 10
moving image 11
moving image 12
moving image 13
moving image 14
moving image 15
moving image 16
moving image 17
moving image 18
moving image 19
moving image 20
moving image 21
moving image 22
moving image 23
moving image 24
moving image 25
moving image 26
moving image 27
moving image 28
moving image 29
moving image 30
moving image 31
moving image 32
moving image 33
moving image 34
moving image 35
moving image 36
moving image 37
moving image 38
moving image 39
moving image 40
moving image 41
moving image 42
moving image 43
moving image 44
moving image 45
moving image 46
moving image 47
moving image 48
moving image 49
moving image 50
moving image 51
moving image 52
moving image 53
moving image 54
moving image 55
moving image 56
moving image 57
moving image 58
moving image 59
moving image 60
moving image 61
moving image 62
mo

**Create image folder with infiltration as positive finding.**

In [ ]:
#Exporting images into new folder
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Infiltration" and row['View Position']== 'PA':
        cnt += 1
print(cnt)

1555


**1555 inflitrate images with PA view. Will create a folder to review.** 

In [ ]:
#Create image folder with infiltrate as positive finding
TARGET_DIR= 'Dataset/Positive'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

CXR folder created


In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Infiltration" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

moving image 0
moving image 1
moving image 2
moving image 3
moving image 4
moving image 5
moving image 6
moving image 7
moving image 8
moving image 9
moving image 10
moving image 11
moving image 12
moving image 13
moving image 14
moving image 15
moving image 16
moving image 17
moving image 18
moving image 19
moving image 20
moving image 21
moving image 22
moving image 23
moving image 24
moving image 25
moving image 26
moving image 27
moving image 28
moving image 29
moving image 30
moving image 31
moving image 32
moving image 33
moving image 34
moving image 35
moving image 36
moving image 37
moving image 38
moving image 39
moving image 40
moving image 41
moving image 42
moving image 43
moving image 44
moving image 45
moving image 46
moving image 47
moving image 48
moving image 49
moving image 50
moving image 51
moving image 52
moving image 53
moving image 54
moving image 55
moving image 56
moving image 57
moving image 58
moving image 59
moving image 60
moving image 61
moving image 62
mo

**Review of the Image Folder:**

**The "No Finding" folder had poor data integrity. There a significant number of films mislabeled (positive findings instead of no findings); included both AP and lateral; inaccurate ages bases on xray findings (skeletally immature or advanced degenerative changes with bone demineralization); poor diagnostic quality of the study. The images were re-read and placed in a new folder was create labeled No_findings_clean.**

**Review of infiltrate file again consisted of poor data integrity. Majority of images had no pulmonary findings. The images were re-read and plaed in a new folder labeled Infiltrate_clean. Since only a small percentage could be used, more folders were created with "Consolidation" and "Pneumonia" labels used for selection.**  
    

In [ ]:
TARGET_DIR= 'Dataset/Positive_consolidation'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

CXR folder created


In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Consolidation" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

moving image 0
moving image 1
moving image 2
moving image 3
moving image 4
moving image 5
moving image 6
moving image 7
moving image 8
moving image 9
moving image 10
moving image 11
moving image 12
moving image 13
moving image 14
moving image 15
moving image 16
moving image 17
moving image 18
moving image 19
moving image 20
moving image 21
moving image 22
moving image 23
moving image 24
moving image 25
moving image 26
moving image 27
moving image 28
moving image 29
moving image 30
moving image 31
moving image 32
moving image 33
moving image 34
moving image 35
moving image 36
moving image 37
moving image 38
moving image 39
moving image 40
moving image 41
moving image 42
moving image 43
moving image 44
moving image 45
moving image 46
moving image 47
moving image 48
moving image 49
moving image 50
moving image 51
moving image 52
moving image 53
moving image 54
moving image 55
moving image 56
moving image 57
moving image 58
moving image 59
moving image 60
moving image 61
moving image 62
mo

In [ ]:
TARGET_DIR= 'Dataset/Positive_pneumonia'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

CXR folder created


In [ ]:
cnt= 0 

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Pneumonia" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path = os.path.join(TARGET_DIR, Image_Index) 
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt += 1

moving image 0
moving image 1
moving image 2
moving image 3
moving image 4
moving image 5
moving image 6
moving image 7
moving image 8
moving image 9
moving image 10
moving image 11
moving image 12
moving image 13
moving image 14
moving image 15
moving image 16
moving image 17
moving image 18
moving image 19
moving image 20
moving image 21
moving image 22
moving image 23
moving image 24
moving image 25
moving image 26
moving image 27
moving image 28
moving image 29
moving image 30
moving image 31
moving image 32
moving image 33
moving image 34
moving image 35
moving image 36
moving image 37
moving image 38
moving image 39
moving image 40
moving image 41
moving image 42
moving image 43
moving image 44
moving image 45
moving image 46
moving image 47
moving image 48
moving image 49
moving image 50
moving image 51
moving image 52
moving image 53
moving image 54
moving image 55
moving image 56
moving image 57
moving image 58


In [ ]:
TARGET_DIR= 'Dataset/Negative_clean'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

CXR folder created


In [ ]:
TARGET_DIR= 'Dataset/Negative_clean'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
#Create smaller dataset to upload of cleaned no findings
files_list = []

for root, dirs, files in os.walk(r"C:\Users\Matrix BR\Dataset\Negative_clean"):
    for file in files:
        if  file.endswith(".png"):
            files_list.append(os.path.join(root, file))

files_normal = random.sample(files_list, 1200) 

destPath = r"C:\Users\Matrix BR\Dataset\oversample_1200\Negative"  

if os.path.isdir(destPath) == False:
        os.makedirs(destPath)

# iteraate over all random files and copy them
for file in files_normal:
    shutil.copy(file, destPath)

print(len(files_normal))

1200


**New folder was created. Now we will make an new CSV file for the 1200 images**

In [ ]:
Negative_images = 'Dataset/oversample_1200/Negative'
file_CXR = os.listdir(Negative_images)


In [ ]:
Negative_images = df[(df["Image Index"].isin(file_CXR))]

In [ ]:
Negative_images.count()

Image Index                    1200
Finding Labels                 1200
Follow-up #                    1200
Patient ID                     1200
Patient Age                    1200
Patient Gender                 1200
View Position                  1200
OriginalImage[Width            1200
Height]                        1200
OriginalImagePixelSpacing[x    1200
y]                             1200
Unnamed: 11                       0
dtype: int64

In [ ]:
Negative_images['Finding Labels'].value_counts()
#the 15 fils label infiltrate had no findings but were labelled "Infiltration" and were moved to the negative file 

No Finding      1185
Infiltration      15
Name: Finding Labels, dtype: int64

In [ ]:
Negative_CXR = Negative_images.replace(['Infiltration','No Finding'], 'Negative')
Negative_CXR['Finding Labels'].value_counts()

Negative    1200
Name: Finding Labels, dtype: int64

In [ ]:
#create new csv
Negative_CXR.to_csv('Negative_CXR.csv')

In [ ]:
Negative_CXR= pd.read_csv("Negative_CXR.csv")
Negative_CXR

,Unnamed: 0,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,777,00000190_000.png,Negative,0,190,30,M,PA,2500,2048,0.168000,0.168000,NaN
1,782,00000190_005.png,Negative,5,190,34,M,PA,2500,2048,0.168000,0.168000,NaN
2,839,00000206_000.png,Negative,0,206,30,M,PA,2048,2500,0.171000,0.171000,NaN
3,840,00000206_001.png,Negative,1,206,30,M,PA,2500,2048,0.171000,0.171000,NaN
4,896,00000213_005.png,Negative,5,213,39,M,PA,2992,2991,0.143000,0.143000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,111966,00030696_000.png,Negative,0,30696,24,M,PA,2021,2021,0.194311,0.194311,NaN
1196,112008,00030729_000.png,Negative,0,30729,35,M,PA,2021,2021,0.194311,0.194311,NaN
1197,112013,00030733_001.png,Negative,1,30733,26,F,AP,3056,2544,0.139000,0.139000,NaN
1198,112025,00030740_000.png,Negative,0,30740,23,M,AP,3056,2544,0.139000,0.139000,NaN


**A folder was created of 398 images.**

In [ ]:
TARGET_DIR= 'Dataset/Positive_clean'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
Positive_images = 'Dataset/Positive_clean'
file_CXR = os.listdir(Positive_images)


In [ ]:
Positive_clean = df[(df["Image Index"].isin(file_CXR))]

In [ ]:
Positive_clean.count()

Image Index       398
Finding Labels    398
Follow-up #       398
Patient Age       398
Patient Gender    398
View Position     398
dtype: int64

In [ ]:
Positive_clean['Finding Labels'].value_counts()

Infiltration     371
No Finding        20
Consolidation      7
Name: Finding Labels, dtype: int64

In [ ]:
Positive_clean = Positive_clean.replace(['Infiltration','No Finding', 'Consolidation'], 'Positive')
Positive_clean['Finding Labels'].value_counts()

Positive    398
Name: Finding Labels, dtype: int64

In [ ]:
Positive_clean.to_csv('Positive_clean.csv')

In [ ]:
Positive_clean

,Image Index,Finding Labels,Follow-up #,Patient Age,Patient Gender,View Position
98,00000022_000.png,Positive,0,48,M,PA
371,00000092_000.png,Positive,0,54,F,PA
502,00000118_008.png,Positive,8,73,M,PA
506,00000118_012.png,Positive,12,73,M,PA
507,00000118_013.png,Positive,13,73,M,PA
...,...,...,...,...,...,...
109796,00029866_009.png,Positive,9,57,F,PA
109806,00029866_019.png,Positive,19,57,F,PA
110283,00030034_000.png,Positive,0,35,M,PA
111851,00030637_022.png,Positive,22,49,M,PA


Now I will create and a dataset split with oversampling of the Positive set. This will help increase the variation of different abnormal findings to better train the model.

In [ ]:

splitfolders.fixed('Dataset/oversample_1200', output="Dataset/Split_1200", seed=1337, oversample=True ) 

Copying files: 1601 files [00:02, 757.61 files/s]
Oversampling: 2 classes [00:04,  2.02s/ classes]


**Folder of oversample positive images and of the 2500-image negative folder.**

In [ ]:

splitfolders.fixed('Dataset/oversample_2500', output="Dataset/Split_2500", seed=1337, oversample=True ) 


Copying files: 0 files [00:00, ? files/s]
Copying files: 1 files [00:00,  8.70 files/s]
Copying files: 19 files [00:00, 12.18 files/s]
Copying files: 37 files [00:00, 16.90 files/s]
Copying files: 55 files [00:00, 23.20 files/s]
Copying files: 74 files [00:00, 31.43 files/s]
Copying files: 92 files [00:00, 41.71 files/s]
Copying files: 110 files [00:00, 54.08 files/s]
Copying files: 128 files [00:00, 68.18 files/s]
Copying files: 146 files [00:00, 83.52 files/s]
Copying files: 165 files [00:01, 99.74 files/s]
Copying files: 184 files [00:01, 116.04 files/s]
Copying files: 202 files [00:01, 123.88 files/s]
Copying files: 219 files [00:01, 92.93 files/s] 
Copying files: 236 files [00:01, 107.37 files/s]
Copying files: 251 files [00:01, 107.82 files/s]
Copying files: 266 files [00:01, 117.34 files/s]
Copying files: 285 files [00:02, 128.84 files/s]
Copying files: 303 files [00:02, 140.43 files/s]
Copying files: 326 files [00:02, 154.22 files/s]
Copying files: 343 files [00:02, 157.40 fil

**Create a new dataset folder of uncleaned images of positive and negative images.**  

In [ ]:
# Create new file of re-read xrays of dirty positive findings
TARGET_DIR= 'Dataset/Junk_positive'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

In [ ]:
cnt=0

for (i, row) in adult_20_40.iterrows():
    if row["Finding Labels"]== "Infiltration" and row['View Position']== 'PA':
        Image_Index= row['Image Index']
        image_path= os.path.join(Images_PATH, Image_Index)
        image_copy_path= os.path.join(TARGET_DIR, Image_Index)
        shutil.copy2(image_path, image_copy_path)
        print("moving image", cnt)
        cnt+= 1

moving image 0
moving image 1
moving image 2
moving image 3
moving image 4
moving image 5
moving image 6
moving image 7
moving image 8
moving image 9
moving image 10
moving image 11
moving image 12
moving image 13
moving image 14
moving image 15
moving image 16
moving image 17
moving image 18
moving image 19
moving image 20
moving image 21
moving image 22
moving image 23
moving image 24
moving image 25
moving image 26
moving image 27
moving image 28
moving image 29
moving image 30
moving image 31
moving image 32
moving image 33
moving image 34
moving image 35
moving image 36
moving image 37
moving image 38
moving image 39
moving image 40
moving image 41
moving image 42
moving image 43
moving image 44
moving image 45
moving image 46
moving image 47
moving image 48
moving image 49
moving image 50
moving image 51
moving image 52
moving image 53
moving image 54
moving image 55
moving image 56
moving image 57
moving image 58
moving image 59
moving image 60
moving image 61
moving image 62
mo

In [ ]:
TARGET_DIR= 'Dataset/Junk_negative'

if not os.path.exists(TARGET_DIR):
    os.mkdir(TARGET_DIR)
    print("CXR folder created")

CXR folder created


**Creat negative dataset folder by randomly selecting 1555 images from the Normal dataset (an extra folder was created prior for this purpose).** 

In [ ]:
#Create smaller dataset of non cleaned no findings
files_list = []

for root, dirs, files in os.walk(r"C:\Users\Matrix BR\Dataset\Normal"):
    for file in files:
        if  file.endswith(".png"):
            files_list.append(os.path.join(root, file))

files_normal = random.sample(files_list, 1555) 

destPath = r"C:\Users\Matrix BR\Dataset\Junk_images\Junk_negative"  

if os.path.isdir(destPath) == False:
        os.makedirs(destPath)

# iteraate over all random files and move them
for file in files_normal:
    shutil.copy(file, destPath)
    
print(len(files_normal))

1555


**Create ultra_clean image dataset with oversampled positive set:
Good study quality with little artifact on images (lines, hardware, leads, catheters).**  

The best 566 images of the cleanest positive findings were selected from "No Finding", "Infiltrate", "Consolidation" and "Pneumonia" files to create the "Positive" dataset. 
1200 of the cleanest negative images were selected from the "No Finding" file for the "Negative" dataset 

In [ ]:

splitfolders.fixed('Dataset/Ultra_clean', output="Dataset/Ultra_clean_1200", seed=1337, oversample=True ) 


Copying files: 0 files [00:00, ? files/s]
Copying files: 7 files [00:00, 67.93 files/s]
Copying files: 25 files [00:00, 82.02 files/s]
Copying files: 42 files [00:00, 96.75 files/s]
Copying files: 60 files [00:00, 109.27 files/s]
Copying files: 76 files [00:00, 119.97 files/s]
Copying files: 94 files [00:00, 129.29 files/s]
Copying files: 110 files [00:00, 136.62 files/s]
Copying files: 125 files [00:00, 135.07 files/s]
Copying files: 139 files [00:00, 134.70 files/s]
Copying files: 154 files [00:01, 137.44 files/s]
Copying files: 168 files [00:01, 137.01 files/s]
Copying files: 183 files [00:01, 138.32 files/s]
Copying files: 197 files [00:01, 138.38 files/s]
Copying files: 212 files [00:01, 139.55 files/s]
Copying files: 227 files [00:01, 141.76 files/s]
Copying files: 242 files [00:01, 143.18 files/s]
Copying files: 258 files [00:01, 144.79 files/s]
Copying files: 274 files [00:01, 148.75 files/s]
Copying files: 289 files [00:02, 149.02 files/s]
Copying files: 304 files [00:02, 148

**The datasets are now completed and will be used to train a simple CNN model.**